# Translating a Decision Problem to an Optimization Model

In the first video we discussed a few key concepts that are necessary for mathematical optimization:
- parameters 
- sets
- decision variables
- constraints
- objective function

In this first modeling example we will see how these are used to formulate a decision problem as an optimization model and code the formulation using `gurobipy`. For more information on all of the commands in the Python API check out our [documentation](https://www.gurobi.com/documentation/10.0/refman/py_python_api_details.html).

https://www.gurobi.com/documentation/10.0/refman/pi.html



## The Decision Problem
We make widgets. Have a set of production facilities that produce boxes of widgets. 
There is also a set of distribution locations that will then distribute the widgets for sale.
Each distribution center has a forecasted demand
and each production facility has a min and max number of widgets it can make during this period.
 We need to ensure that each distribution facility receives enough widgets to satisfy demand from production
 
 obj and we want to do this at minimal cost. The minimum production is 75% of the production facilities max value.

## Sets and Define Model
Our sets are:
- $P = \{\texttt{'Baltimore', 'Cleveland', 'Little Rock', 'Birmingham', 'Charleston'}\} \quad\quad\quad\quad\quad\quad\quad\space\space \texttt{production}$
- $D = \{\texttt{'Columbia', 'Indianapolis', 'Lexington', 'Nashville', 'Richmond', 'St. Louis'}\} \quad\quad\quad \texttt{distribution}$

To index each set, we'll use the lowercase letter of each set. Letters used for sets and indices are up to you. Typically, capital letters are for sets and corresponding lowercase will be the index. Single letters are used mainly for conciseness.

In [21]:
#%pip install gurobipy
#!conda install pandas

# First, import packages
import pandas as pd
import gurobipy as gp
from gurobipy import GRB

#Turn Off warnnings
import warnings
warnings.filterwarnings('ignore')

# Sets P and D, respectively
# When we code sets we can be more descriptive in the name
production = ['Baltimore','Cleveland','Little Rock','Birmingham','Charleston']
distribution = ['Columbia','Indianapolis','Lexington','Nashville','Richmond','St. Louis']
print (type(production))
print (type(distribution))
# Define a gurobipy model for the decision problem
m = gp.Model('widgets')

<class 'list'>
<class 'list'>


## Parameters

Parameters of a math optimization problem are values treated as constants in the model and are associated with the decision variables. For this decision problem these values are the limits of each production facility, the demand for each distribution center, and the pairwise costs between production and distribution locations. 

- $m_p$ is the max production in location $p$, $\forall p \in P \quad\quad\quad\quad\quad\quad\quad\quad\quad\quad\quad\quad\quad\quad\space\space \texttt{max}\_\texttt{prod[p]}$
- $n_d$ is the number of customers for a distribution center $d$, $\forall d \in D \quad\quad\quad\quad\quad\quad\quad\quad \texttt{n}\_\texttt{demand[d]}$
- $c_{p,d}$ is the cost to ship a widget between location $p$ and location $d$, $\forall p \in P, d \in D \quad\quad\quad \texttt{cost[p,d]}$

Read m sub p

In [22]:
# Use squeeze=True to make the costs a series, read in a csv 

transp_cost = pd.read_csv( 'cost.csv', index_col=[0,1], squeeze=True)
print (transp_cost)
print(transp_cost)
# Pivot to view the costs a bit easier
transp_cost.reset_index().pivot(index='production', columns='distribution', values='cost') # just visual perspective 

production   distribution
Baltimore    Columbia        4.50
             Indianapolis    5.09
             Lexington       4.33
             Nashville       5.96
             Richmond        1.96
             St. Louis       7.30
Cleveland    Columbia        2.43
             Indianapolis    2.37
             Lexington       2.54
             Nashville       4.13
             Richmond        3.20
             St. Louis       4.88
Little Rock  Columbia        6.42
             Indianapolis    4.83
             Lexington       3.39
             Nashville       4.40
             Richmond        7.44
             St. Louis       2.92
Birmingham   Columbia        3.33
             Indianapolis    4.33
             Lexington       3.38
             Nashville       1.53
             Richmond        5.95
             St. Louis       4.01
Charleston   Columbia        3.02
             Indianapolis    2.61
             Lexington       1.61
             Nashville       4.44
             Richmond 

distribution,Columbia,Indianapolis,Lexington,Nashville,Richmond,St. Louis
production,,,,,,
Baltimore,4.50,5.09,4.33,5.96,1.96,7.30
Birmingham,3.33,4.33,3.38,1.53,5.95,4.01
Charleston,3.02,2.61,1.61,4.44,2.36,4.60
Cleveland,2.43,2.37,2.54,4.13,3.20,4.88
Little Rock,6.42,4.83,3.39,4.40,7.44,2.92


In [23]:
max_prod = pd.Series([180,200,140,80,180], index = production, name = "max_production")
n_demand = pd.Series([89,95,121,101,116,181], index = distribution, name = "demand") 
max_prod.to_frame()
#n_demand.to_frame()

,max_production
Baltimore,180
Cleveland,200
Little Rock,140
Birmingham,80
Charleston,180


We also have the requirement that each production facility needs to produce at 75% of this maximum output. We'll denote this value by $a$ in the formulation "frac" for the fraction of maximum production required. Initially we set $a = 0.75$.

In [24]:
frac = 0.75

## Decision Variables  20 min
This is what the optimization solver determines, which are the actions you have control over. As a reminder, they come in three main flavors:
- `Continuous`: Price of a product
- `Integer`: The number of food trucks to use for an event
- `Binary`: Yes/no decision to include a certain stock in a portfolio

Decision variables (and parameters) are indexed using elements of sets that we define for the problem. In this example, let's start with a set of cities that produce our widget, which we call set $P$ for the formulation but can define as 'production' in the code. And a set of cities that distribute the widget $D$ and 'distribution' similarly. The decision here is to determine the number of boxes to send from each production facility to each distribution location. 

Let $x_{p,d}$ be the number of widgets that are produced at facility $i$ and shipped to location $j$.

### Add Variables in gurobipy
`gurobipy` let's you add decision variables primarily with two (similar) commands:
- [addVar()](https://www.gurobi.com/documentation/10.0/refman/py_model_addvar.html) adds a single variable
- [addVars()](https://www.gurobi.com/documentation/10.0/refman/py_model_addvar.html) adds a group of variables by sets/indices

When using `addVars` you have to provide the indices of the variables you want to add, which for us are the production and distribution locations. There are other arguments we can use and will cover a couple of them later on.  

### Our Decision Variables
As is often the case in writing code, there are several ways to get to the same point. Below we can see three different ways to create the decision variables. 

In [25]:
# loop through each p and d combination to create a decision variable
m = gp.Model('widgets')
x = {}
for p in production:
    for d in distribution:
        x[p,d] = m.addVar(name = p+"_to_"+d)
m.update()
x

{('Baltimore', 'Columbia'): <gurobi.Var Baltimore_to_Columbia>,
 ('Baltimore', 'Indianapolis'): <gurobi.Var Baltimore_to_Indianapolis>,
 ('Baltimore', 'Lexington'): <gurobi.Var Baltimore_to_Lexington>,
 ('Baltimore', 'Nashville'): <gurobi.Var Baltimore_to_Nashville>,
 ('Baltimore', 'Richmond'): <gurobi.Var Baltimore_to_Richmond>,
 ('Baltimore', 'St. Louis'): <gurobi.Var Baltimore_to_St. Louis>,
 ('Cleveland', 'Columbia'): <gurobi.Var Cleveland_to_Columbia>,
 ('Cleveland', 'Indianapolis'): <gurobi.Var Cleveland_to_Indianapolis>,
 ('Cleveland', 'Lexington'): <gurobi.Var Cleveland_to_Lexington>,
 ('Cleveland', 'Nashville'): <gurobi.Var Cleveland_to_Nashville>,
 ('Cleveland', 'Richmond'): <gurobi.Var Cleveland_to_Richmond>,
 ('Cleveland', 'St. Louis'): <gurobi.Var Cleveland_to_St. Louis>,
 ('Little Rock', 'Columbia'): <gurobi.Var Little Rock_to_Columbia>,
 ('Little Rock', 'Indianapolis'): <gurobi.Var Little Rock_to_Indianapolis>,
 ('Little Rock', 'Lexington'): <gurobi.Var Little Rock_to_Le

In [26]:
# Provide each set for the indices 
m = gp.Model('widgets')
x = m.addVars(production, distribution, name = 'prod_ship') # Give gurobi th set you want create dicsion variable for 
# pord_ship add the text of indices to the set 
m.update()
x

{('Baltimore', 'Columbia'): <gurobi.Var prod_ship[Baltimore,Columbia]>,
 ('Baltimore', 'Indianapolis'): <gurobi.Var prod_ship[Baltimore,Indianapolis]>,
 ('Baltimore', 'Lexington'): <gurobi.Var prod_ship[Baltimore,Lexington]>,
 ('Baltimore', 'Nashville'): <gurobi.Var prod_ship[Baltimore,Nashville]>,
 ('Baltimore', 'Richmond'): <gurobi.Var prod_ship[Baltimore,Richmond]>,
 ('Baltimore', 'St. Louis'): <gurobi.Var prod_ship[Baltimore,St. Louis]>,
 ('Cleveland', 'Columbia'): <gurobi.Var prod_ship[Cleveland,Columbia]>,
 ('Cleveland', 'Indianapolis'): <gurobi.Var prod_ship[Cleveland,Indianapolis]>,
 ('Cleveland', 'Lexington'): <gurobi.Var prod_ship[Cleveland,Lexington]>,
 ('Cleveland', 'Nashville'): <gurobi.Var prod_ship[Cleveland,Nashville]>,
 ('Cleveland', 'Richmond'): <gurobi.Var prod_ship[Cleveland,Richmond]>,
 ('Cleveland', 'St. Louis'): <gurobi.Var prod_ship[Cleveland,St. Louis]>,
 ('Little Rock', 'Columbia'): <gurobi.Var prod_ship[Little Rock,Columbia]>,
 ('Little Rock',
  'Indianapolis

In [27]:
# The index of the tranporation costs have each combination of production and distribution location
m = gp.Model('widgets')
print(transp_cost)
print(transp_cost.index)
x = m.addVars(transp_cost.index, name = 'prod_ship') # we have already the replatioshiping in  the cost data 
m.update()
x

production   distribution
Baltimore    Columbia        4.50
             Indianapolis    5.09
             Lexington       4.33
             Nashville       5.96
             Richmond        1.96
             St. Louis       7.30
Cleveland    Columbia        2.43
             Indianapolis    2.37
             Lexington       2.54
             Nashville       4.13
             Richmond        3.20
             St. Louis       4.88
Little Rock  Columbia        6.42
             Indianapolis    4.83
             Lexington       3.39
             Nashville       4.40
             Richmond        7.44
             St. Louis       2.92
Birmingham   Columbia        3.33
             Indianapolis    4.33
             Lexington       3.38
             Nashville       1.53
             Richmond        5.95
             St. Louis       4.01
Charleston   Columbia        3.02
             Indianapolis    2.61
             Lexington       1.61
             Nashville       4.44
             Richmond 

{('Baltimore', 'Columbia'): <gurobi.Var prod_ship[Baltimore,Columbia]>,
 ('Baltimore', 'Indianapolis'): <gurobi.Var prod_ship[Baltimore,Indianapolis]>,
 ('Baltimore', 'Lexington'): <gurobi.Var prod_ship[Baltimore,Lexington]>,
 ('Baltimore', 'Nashville'): <gurobi.Var prod_ship[Baltimore,Nashville]>,
 ('Baltimore', 'Richmond'): <gurobi.Var prod_ship[Baltimore,Richmond]>,
 ('Baltimore', 'St. Louis'): <gurobi.Var prod_ship[Baltimore,St. Louis]>,
 ('Cleveland', 'Columbia'): <gurobi.Var prod_ship[Cleveland,Columbia]>,
 ('Cleveland', 'Indianapolis'): <gurobi.Var prod_ship[Cleveland,Indianapolis]>,
 ('Cleveland', 'Lexington'): <gurobi.Var prod_ship[Cleveland,Lexington]>,
 ('Cleveland', 'Nashville'): <gurobi.Var prod_ship[Cleveland,Nashville]>,
 ('Cleveland', 'Richmond'): <gurobi.Var prod_ship[Cleveland,Richmond]>,
 ('Cleveland', 'St. Louis'): <gurobi.Var prod_ship[Cleveland,St. Louis]>,
 ('Little Rock', 'Columbia'): <gurobi.Var prod_ship[Little Rock,Columbia]>,
 ('Little Rock',
  'Indianapolis

The command `m.update()` updates the model to include any changes that have been made, like adding variables. It doesn't need to be run in every cell but if you see *Awaiting Model Update* in the output of a cell, then this should prevent that from happening.

## Constraints
We outlined production and demand constraints at the beginning of this example; now we formulate and code them. Note that it doesn't matter the order in which constraints (and/or decision variables) are added to the model.

### Add Constraints in gurobipy
Adding constraints to a model is similar to adding variables:
- [addConstr()](https://www.gurobi.com/documentation/10.0/refman/py_model_addconstr.html) adds a single constraint
- [addConstrs()](https://www.gurobi.com/documentation/10.0/refman/py_model_addconstrs.htmll) adds a group of constraints using a Python `generator` expression
 
### Our Constraints
To start, we'll formulate the demand constraints for each distribution location first and add them to the model. # add bounch of them 

\begin{align*} 
\sum_{p}x_{p,d} \ge n_d, \quad \forall d \in D \quad\quad \texttt{meet}\_\texttt{demand[d]}\\ 
\end{align*}

sum over p , every thing send to 

**This will be the first time we use [gp.quicksum()](https://www.gurobi.com/documentation/10.0/refman/py_quicksum.html). There are other ways to sum expressions in gurobipy and while this method isn't the most concise to code, it is easy to compare it to the summation in the formulation to see how it works.**

In [52]:
# lets model the contarint above, I can do the first they can do the rest 
meet_demand =m.addConstrs((gp.quicksum(x[p,d] for p in production ) >= n_demand[d] for d in distribution),name= 'meet_demand') 
print(type(x))

#gp.quicksum(x[p,d] for p in production)
meet_demand = m.addConstrs((gp.quicksum(x[p,d] for p in production) >= n_demand[d] for d in distribution), name = 'meet_demand')
#meet_demand = m.addConstrs(x.sum('*', j) >= demand[j] for j in distribution)
m.update()
meet_demand

GurobiError: Variable not in model

Next we have the maximum number of widgets each production facility can make. We also have that each facility must make at least 75% of its max production. 

$$
\begin{align*} 
\sum_{d}x_{p,d} &\le m_p, &\forall p \in P \quad\quad &\texttt{can}\_\texttt{produce[p]}\\ 
\sum_{d}x_{p,d} &\ge a*m_p,&\forall p \in P \quad\quad &\texttt{must}\_\texttt{produce[p]}\\ 
\end{align*}
$$


In [32]:
can_produce = m.addConstrs((gp.quicksum(x[p,d] for d in distribution) <= max_prod[p] for p in production), name = 'can_produce')# (x.sum(i, '*')
must_produce = m.addConstrs((gp.quicksum(x[p,d] for d in distribution) >= frac*max_prod[p] for p in production), name = 'must_produce')
m.update()
can_produce #wuick sum genrator  the desicon varaibe we can sum over 

{'Baltimore': <gurobi.Constr can_produce[Baltimore]>,
 'Cleveland': <gurobi.Constr can_produce[Cleveland]>,
 'Little Rock': <gurobi.Constr can_produce[Little Rock]>,
 'Birmingham': <gurobi.Constr can_produce[Birmingham]>,
 'Charleston': <gurobi.Constr can_produce[Charleston]>}

## Objective Function
We were told to **reduce** the transportation costs and we'll use this to determine our objective function as minimizing the total cost to ship widgets from production to distribution locations. 

### Setting the Objective in gurobipy
This is done using [setObjective()](https://www.gurobi.com/documentation/10.0/refman/py_model_setobjective.html). The second argument (in this case `GRB.MINIMIZE`) is called the model's *sense*. For a maximization problem we would use `GRB.MAXIMIZE`. 

### Our Objective Function
\begin{align*} 
{\rm minimize} \space \sum_{p,d}c_{p,d}x_{p,d}, \quad \forall p \in P, d \in D\\ 
\end{align*}

above is the notation 
assoaciation cost 

In [33]:
m.setObjective(gp.quicksum(transp_cost[i,j]*x[i,j] for i in production for j in distribution), GRB.MINIMIZE) 

## Find, Extract, and Analyze the Solution
Before running the optimization, it is a good idea to write an `lp` file. This is a text file that prints out the variables, constraints, and object like we would see in the *formulation*, just without the summation symbols and using the names we designated.

In [34]:
m.write('widget_shipment.lp')
m.write('widget_shipment.mps')
m.write('widget_shipment.rew')

#send to suppport 

#later in the tuning 

### Run the Optimization

In [35]:
m.optimize()

Gurobi Optimizer version 10.0.0 build v10.0.0rc2 (win64)

CPU model: Intel(R) Core(TM) i7-1065G7 CPU @ 1.30GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 22 rows, 30 columns and 120 nonzeros
Model fingerprint: 0x3b8b44d3
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [2e+00, 7e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [6e+01, 2e+02]
Presolve removed 11 rows and 0 columns
Presolve time: 0.01s
Presolved: 11 rows, 35 columns, 65 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    0.0000000e+00   1.610000e+02   0.000000e+00      0s
      15    1.7048900e+03   0.000000e+00   0.000000e+00      0s

Solved in 15 iterations and 0.01 seconds (0.00 work units)
Optimal objective  1.704890000e+03


### Extract the Solution
There are many ways to get the values of decision variables out of gurobipy. 
https://www.gurobi.com/documentation/10.0/refman/attributes.html
https://www.gurobi.com/documentation/10.0/refman/attribute_examples.html#sec:AttributeExamples

In [36]:
# 40 min 
# 
# With the X attribute, you access the variable's value in the current solution. Note that the code first does
# https://www.gurobi.com/documentation/10.0/refman/x.html
print(type(x))
print(x) # not the value 

print(m.numintvars)
print(m.getAttr("numIntVars"))
print(m.getAttr(GRB.Attr.NumIntVars))
print(m.getAttr("X", m.getVars()))
print(m.getAttr("Pi", m.getConstrs()))


<class 'gurobipy.tupledict'>
{('Baltimore', 'Columbia'): <gurobi.Var prod_ship[Baltimore,Columbia] (value 0.0)>, ('Baltimore', 'Indianapolis'): <gurobi.Var prod_ship[Baltimore,Indianapolis] (value 0.0)>, ('Baltimore', 'Lexington'): <gurobi.Var prod_ship[Baltimore,Lexington] (value 0.0)>, ('Baltimore', 'Nashville'): <gurobi.Var prod_ship[Baltimore,Nashville] (value 19.0)>, ('Baltimore', 'Richmond'): <gurobi.Var prod_ship[Baltimore,Richmond] (value 116.0)>, ('Baltimore', 'St. Louis'): <gurobi.Var prod_ship[Baltimore,St. Louis] (value 0.0)>, ('Cleveland', 'Columbia'): <gurobi.Var prod_ship[Cleveland,Columbia] (value 89.0)>, ('Cleveland', 'Indianapolis'): <gurobi.Var prod_ship[Cleveland,Indianapolis] (value 95.0)>, ('Cleveland', 'Lexington'): <gurobi.Var prod_ship[Cleveland,Lexington] (value 0.0)>, ('Cleveland', 'Nashville'): <gurobi.Var prod_ship[Cleveland,Nashville] (value 2.0)>, ('Cleveland', 'Richmond'): <gurobi.Var prod_ship[Cleveland,Richmond] (value 0.0)>, ('Cleveland', 'St. Louis')

In [37]:
print(transp_cost.index)

x_values = pd.Series(m.getAttr('X', x), name = "shipment", index = transp_cost.index)
sol = pd.concat([transp_cost, x_values], axis=1)
sol 
#sol[sol.shipment > 0]

MultiIndex([(  'Baltimore',     'Columbia'),
            (  'Baltimore', 'Indianapolis'),
            (  'Baltimore',    'Lexington'),
            (  'Baltimore',    'Nashville'),
            (  'Baltimore',     'Richmond'),
            (  'Baltimore',    'St. Louis'),
            (  'Cleveland',     'Columbia'),
            (  'Cleveland', 'Indianapolis'),
            (  'Cleveland',    'Lexington'),
            (  'Cleveland',    'Nashville'),
            (  'Cleveland',     'Richmond'),
            (  'Cleveland',    'St. Louis'),
            ('Little Rock',     'Columbia'),
            ('Little Rock', 'Indianapolis'),
            ('Little Rock',    'Lexington'),
            ('Little Rock',    'Nashville'),
            ('Little Rock',     'Richmond'),
            ('Little Rock',    'St. Louis'),
            ( 'Birmingham',     'Columbia'),
            ( 'Birmingham', 'Indianapolis'),
            ( 'Birmingham',    'Lexington'),
            ( 'Birmingham',    'Nashville'),
          

cost  shipment
production  distribution                
Baltimore   Columbia      4.50       0.0
            Indianapolis  5.09       0.0
            Lexington     4.33       0.0
            Nashville     5.96      19.0
            Richmond      1.96     116.0
            St. Louis     7.30       0.0
Cleveland   Columbia      2.43      89.0
            Indianapolis  2.37      95.0
            Lexington     2.54       0.0
            Nashville     4.13       2.0
            Richmond      3.20       0.0
            St. Louis     4.88       0.0
Little Rock Columbia      6.42       0.0
            Indianapolis  4.83       0.0
            Lexington     3.39       0.0
            Nashville     4.40       0.0
            Richmond      7.44       0.0
            St. Louis     2.92     140.0
Birmingham  Columbia      3.33       0.0
            Indianapolis  4.33       0.0
            Lexington     3.38       0.0
            Nashville     1.53      80.0
            Richmond      5.95       0.0
            St. Louis     4.01       0.0
Charleston  Columbia      3.02       0.0
            Indianapolis  2.61       0.0
            Lexington     1.61     121.0
            Nashville     4.44       0.0
            Richmond      2.36       0.0
            St. Louis     4.60      41.0

In [38]:
m.getAttr('X', x)

{('Baltimore', 'Columbia'): 0.0,
 ('Baltimore', 'Indianapolis'): 0.0,
 ('Baltimore', 'Lexington'): 0.0,
 ('Baltimore', 'Nashville'): 19.0,
 ('Baltimore', 'Richmond'): 116.0,
 ('Baltimore', 'St. Louis'): 0.0,
 ('Cleveland', 'Columbia'): 89.0,
 ('Cleveland', 'Indianapolis'): 95.0,
 ('Cleveland', 'Lexington'): 0.0,
 ('Cleveland', 'Nashville'): 2.0,
 ('Cleveland', 'Richmond'): 0.0,
 ('Cleveland', 'St. Louis'): 0.0,
 ('Little Rock', 'Columbia'): 0.0,
 ('Little Rock', 'Indianapolis'): 0.0,
 ('Little Rock', 'Lexington'): 0.0,
 ('Little Rock', 'Nashville'): 0.0,
 ('Little Rock', 'Richmond'): 0.0,
 ('Little Rock', 'St. Louis'): 140.0,
 ('Birmingham', 'Columbia'): 0.0,
 ('Birmingham', 'Indianapolis'): 0.0,
 ('Birmingham', 'Lexington'): 0.0,
 ('Birmingham', 'Nashville'): 80.0,
 ('Birmingham', 'Richmond'): 0.0,
 ('Birmingham', 'St. Louis'): 0.0,
 ('Charleston', 'Columbia'): 0.0,
 ('Charleston', 'Indianapolis'): 0.0,
 ('Charleston', 'Lexington'): 121.0,
 ('Charleston', 'Nashville'): 0.0,
 ('Charles

Here are a couple of other ways to get the solution.

In [39]:
# You can get the name and value of all the decision variables: if you do not like pandas 
all_vars = {v.varName: v.x for v in m.getVars()} 
print(type(all_vars))
all_vars

<class 'dict'>


{'prod_ship[Baltimore,Columbia]': 0.0,
 'prod_ship[Baltimore,Indianapolis]': 0.0,
 'prod_ship[Baltimore,Lexington]': 0.0,
 'prod_ship[Baltimore,Nashville]': 19.0,
 'prod_ship[Baltimore,Richmond]': 116.0,
 'prod_ship[Baltimore,St. Louis]': 0.0,
 'prod_ship[Cleveland,Columbia]': 89.0,
 'prod_ship[Cleveland,Indianapolis]': 95.0,
 'prod_ship[Cleveland,Lexington]': 0.0,
 'prod_ship[Cleveland,Nashville]': 2.0,
 'prod_ship[Cleveland,Richmond]': 0.0,
 'prod_ship[Cleveland,St. Louis]': 0.0,
 'prod_ship[Little Rock,Columbia]': 0.0,
 'prod_ship[Little Rock,Indianapolis]': 0.0,
 'prod_ship[Little Rock,Lexington]': 0.0,
 'prod_ship[Little Rock,Nashville]': 0.0,
 'prod_ship[Little Rock,Richmond]': 0.0,
 'prod_ship[Little Rock,St. Louis]': 140.0,
 'prod_ship[Birmingham,Columbia]': 0.0,
 'prod_ship[Birmingham,Indianapolis]': 0.0,
 'prod_ship[Birmingham,Lexington]': 0.0,
 'prod_ship[Birmingham,Nashville]': 80.0,
 'prod_ship[Birmingham,Richmond]': 0.0,
 'prod_ship[Birmingham,St. Louis]': 0.0,
 'prod_shi

**Or you can only iterate over a specific variable and only return values that are of interest to you. Remember, x is a dict in python. So, iterate over it, the same way you iterate over any dictionary**

In [15]:
xvals = {k: v.x for k,v in x.items() if v.x > 0} 
xvals 

{('Baltimore', 'Nashville'): 19.0,
 ('Baltimore', 'Richmond'): 116.0,
 ('Cleveland', 'Columbia'): 89.0,
 ('Cleveland', 'Indianapolis'): 95.0,
 ('Cleveland', 'Nashville'): 2.0,
 ('Little Rock', 'St. Louis'): 140.0,
 ('Birmingham', 'Nashville'): 80.0,
 ('Charleston', 'Lexington'): 121.0,
 ('Charleston', 'St. Louis'): 41.0}

In [16]:
x_values.sum(), n_demand.sum()

(703.0, 703)

### Solution Analysis  , contraints attributes 45 min
While determining the optimal transportation of widgets was our goal, we may want to dig a little deeper into the solution. For example we can aggregate the total production by facility to see which locations (if any) did not produce their maximum capacity of widgets and which (if any) production facilities are at the lower bound of their production. 

In [40]:
# Sum the shipment amount by production facility
ship_out = sol.groupby('production')['shipment'].sum()
print(ship_out) # what each porduction is shsippment out 
# let check the utlization of each prodcution 
pd.DataFrame({'Remaining':max_prod - ship_out, 'Utilization':ship_out/max_prod})

production
Baltimore      135.0
Birmingham      80.0
Charleston     162.0
Cleveland      186.0
Little Rock    140.0
Name: shipment, dtype: float64


,Remaining,Utilization
Baltimore,45.0,0.75
Birmingham,0.0,1.00
Charleston,18.0,0.90
Cleveland,14.0,0.93
Little Rock,0.0,1.00


**In mathematical optimization, when the left-hand and right-hand sides of an inequality constraint are equal, we say the constraint is `binding`. When this *doesn't happen* then there is `slack` or `surplus` in that constraint. We can get this value by calling the `Slack` attribute of a constraint.**

In [43]:
# https://www.gurobi.com/documentation/10.0/refman/pi.html
# slack attibute 
print(type(can_produce))
print(can_produce)
pd.DataFrame({'Remaining':[can_produce[p].Slack for p in production], 
              'Utilization':[1-can_produce[p].Slack/max_prod[p] for p in production]}, 
             index = production)

<class 'gurobipy.tupledict'>
{'Baltimore': <gurobi.Constr can_produce[Baltimore]>, 'Cleveland': <gurobi.Constr can_produce[Cleveland]>, 'Little Rock': <gurobi.Constr can_produce[Little Rock]>, 'Birmingham': <gurobi.Constr can_produce[Birmingham]>, 'Charleston': <gurobi.Constr can_produce[Charleston]>}


,Remaining,Utilization
Baltimore,45.0,0.75
Cleveland,14.0,0.93
Little Rock,0.0,1.00
Birmingham,0.0,1.00
Charleston,18.0,0.90


In [45]:
max_prod

Baltimore      180
Cleveland      200
Little Rock    140
Birmingham      80
Charleston     180
Name: max_production, dtype: int64

# Using Binary Variables
As we described in the first session and also at the top of this notebook, binary variables are used to choose alternatives in mathematical optimization. They can be interpreted as a yes/no decision or an on/off switch. 

In the original problem Birmingham's production is much lower than the rest of the facilities. Suppose we have the option to expand that facilities max capacity by either 25 or 50 widgets, but there is a cost of \\$50 and \\$75, respectively, to choose one of these options and we can choose at most one. We'll use a binary decision variable for each option named $xprod$. 

Let $xprod_0 = 1$ if we choose the first option and expand production capacity by 25 and $0$ otherwise.
Let $xprod_1 = 1$ if we choose the second option and expand production capacity by 50 and $0$ otherwise.

While it's fairly common to use single lowercase letters as decision variables, it is not necessary and you'll see variables defined as above (where they are more descriptive) quite often. We will formulate a new model that contains the same decision variables and demand constraints as before.

In [47]:
# We use m2 for the second model
# These parts are the same as above outside of the new model name
m2 = gp.Model('widgets2')
# print (transp_cost) pandas seris 

x = m2.addVars(production, distribution, obj = transp_cost, name = 'prod_ship')
# obj is an optional argument to add the objective coffient Objective coefficient(s) for new variables.
meet_demand = m2.addConstrs((gp.quicksum(x[p,d] for p in production) >= n_demand[d] for d in distribution), name = 'meet_demand')

m2.update()
# print(x)

In the cell above we did use new argument of the `addVars()` function: `obj`. This will set the coefficient of the added decision variables in the objective function and is equivalent to what we did earlier by attaching the transportation costs between each production and distribution location to the appropriate decision variable.  

Next, we'll add the same constraints for production limits as before for each production facility other than Birmingham. The formulation is basically the same other than the set the constraints hold for. 
$$
\begin{align*} 
\sum_{d}x_{p,d} &\le m_p, &\forall p \in P -\{\texttt{Birmingham}\} \\ 
\sum_{d}x_{p,d} &\ge a*m_p,&\forall p \in P -\{\texttt{Birmingham}\} \\ 
\end{align*}
$$

In gurobipy, this is done by adding a condition in the `generator` expression. 

**In Python, to create iterators, we can use both regular functions and generators. Generators are written just like a normal function but we use yield() instead of return() for returning a result. It is more powerful as a tool to implement iterators. **

In [51]:
# Python code to illustrate generator, yield() and next().
def generator():
    t = 1
    print ('First result is ',t)
    yield t
    #print(t)
    #print(type(t))
 
    t += 1
    print ('Second result is ',t)
    yield t
 
    t += 1
    print('Third result is ',t)
    yield t
 
call = generator()
next(call)
next(call)
next(call)

First result is  1
Second result is  2
Third result is  3


3

In [20]:
can_produce = m2.addConstrs((gp.quicksum(x[p,d] for d in distribution) <= max_prod[p] for p in production if p != 'Birmingham'), name = 'can_produce')
must_produce =  m2.addConstrs((gp.quicksum(x[p,d] for d in distribution) >= frac*max_prod[p] for p in production if p != 'Birmingham'), name = 'must_produce')

Now, add the new binary variables.

In [21]:
xprod = m2.addVars(range(2), vtype = GRB.BINARY, obj = [50, 75], name = 'expand_Birmingham_prod')

# 60 min 
Let's breakdown each of the arguments in the cell above -- there are a few new things there.
1. `range(n)` is used to add $n$ decision variables. In this case we add 2 variables.
2. We need to declare this as a binary variable using `vtype`.
3. We again use the `obj` capability to immediately set the objective function coefficients for these variables.

The objective and new binary variables look like this in the formulation: 

\begin{align*} 
{\rm minimize} \space &\sum_{p,d}c_{p,d}x_{p,d} + 50*xprod_0 + 75*xprod_1, \quad &\forall p \in P, d \in D\\ 
&xprod_i \in \{0,1\}, &{\rm for} \space i \in \{0,1\}
\end{align*}

Next we have the production constraints that are specific to the Birmingham facility.

$$
\begin{align*} 
\sum_{d}x_{p,d} &\le m_p + 25*xprod_0 + 50*xprod_1, & p = \texttt{Birmingham} \\ 
\sum_{d}x_{p,d} &\ge a*(m_p+ 25*xprod_0 + 50*xprod_1),& p = \texttt{Birmingham} \\ 
\end{align*}
$$

In [22]:
Birmingham_max = m2.addConstr(gp.quicksum(x['Birmingham',d] for d in distribution) <= max_prod['Birmingham'] + 25*xprod[0] + 50*xprod[1], name = 'Birmingham_add_max')
Birmingham_min = m2.addConstr(gp.quicksum(x['Birmingham',d] for d in distribution) >= frac*(max_prod['Birmingham'] + 25*xprod[0] + 50*xprod[1]), name = 'Birmingham_add_min')

It was stated above that we can select at most one of the expansion options which means we cannot allow both $xprod_0$ and $xprod_1$ to equal one. To model this we add a constraint limiting the sum of these two binary variables to at most one.

$$
\begin{align*}
\sum_{i}xprod_i \le 1
\end{align*}
$$
The corresponding constraint in gurobipy:

In [23]:
Birmingham_lim = m2.addConstr(gp.quicksum(xprod[i] for i in range(2)) <= 1, name = 'expansion_choice')

Now we can run this optimization model and see if this potential expansion will help us reduce overall costs. 

In [24]:
m2.optimize()

Gurobi Optimizer version 10.0.0 build v10.0.0rc2 (win64)

CPU model: Intel(R) Core(TM) i7-1065G7 CPU @ 1.30GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 17 rows, 32 columns and 96 nonzeros
Model fingerprint: 0x80ef8db6
Variable types: 30 continuous, 2 integer (2 binary)
Coefficient statistics:
  Matrix range     [1e+00, 5e+01]
  Objective range  [2e+00, 8e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+02]
Presolve time: 0.01s
Presolved: 17 rows, 32 columns, 96 nonzeros
Variable types: 30 continuous, 2 integer (2 binary)
Found heuristic solution: objective 1704.8900000

Root relaxation: objective 1.684260e+03, 14 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 1684.26000    0    1 1704.89000 1684.26000  1.21%    

In [25]:
obj1 = m.getObjective()
obj2 = m2.getObjective()
print(f"The original model had a total cost of {round(obj1.getValue(),2)}")
print(f"The new formualtion has a total cost of {round(obj2.getValue(),2)}")

The original model had a total cost of 1704.89
The new formualtion has a total cost of 1700.4


What does the change in objective function value tell us? 

Let's look at values of our binary variables.

In [26]:
pd.Series(m2.getAttr('X', xprod))

0    1.0
1    0.0
dtype: float64

The model selected the first expansion option since $xprod_0 = 1$, which was increasing production by 25 widgets in Birmingham. We can see the rest of the solution, which will include the increase in Birmingham's production capacity. 

In [27]:
x2_values = pd.Series(m2.getAttr('X', x), name = "shipment", index = transp_cost.index)
sol2 = pd.concat([transp_cost, x2_values], axis=1)
sol2[sol2.shipment > 0]

cost  shipment
production  distribution                
Baltimore   Richmond      1.96     135.0
Cleveland   Columbia      2.43      89.0
            Indianapolis  2.37      95.0
Little Rock St. Louis     2.92     140.0
Birmingham  Nashville     1.53     101.0
            St. Louis     4.01       4.0
Charleston  Lexington     1.61     121.0
            St. Louis     4.60      37.0

### Homework! (not really, but something to look into)
Analyze how the optimal solution changes between the two models. You'll notice something weird. 
- What is it that's odd?
- Why do you think it happened?
- How would you address it from formulation perspective and a business perspective?

# Exercises 70 min 

### Formulation and Coding 
Below is code for the entire original model in one cell if you would like to use it to help with these exercises.

In [28]:
#%pip install gurobipy

import pandas as pd
import gurobipy as gp
from gurobipy import GRB

production = ['Baltimore','Cleveland','Little Rock','Birmingham','Charleston']
distribution = ['Columbia','Indianapolis','Lexington','Nashville','Richmond','St. Louis']

path = 'https://raw.githubusercontent.com/Gurobi/modeling-examples/master/optimization101/Modeling_Session_1/'
transp_cost = pd.read_csv(path + 'cost.csv', index_col=[0,1], squeeze=True)
max_prod = pd.Series([180,200,140,80,180], index = production, name = "max_production")
n_demand = pd.Series([89,95,121,101,116,181], index = distribution, name = "demand") 
frac = 0.75

m = gp.Model('widgets')
x = m.addVars(production, distribution, name = 'prod_ship')
meet_demand = m.addConstrs((gp.quicksum(x[p,d] for p in production) >= n_demand[d] for d in distribution), name = 'meet_demand')
can_produce = m.addConstrs((gp.quicksum(x[p,d] for d in distribution) <= max_prod[p] for p in production), name = 'can_produce')
must_produce = m.addConstrs((gp.quicksum(x[p,d] for d in distribution) >= frac*max_prod[p] for p in production), name = 'must_produce')
m.setObjective(gp.quicksum(transp_cost[i,j]*x[i,j] for i in production for j in distribution), GRB.MINIMIZE) 
m.optimize()

Gurobi Optimizer version 10.0.0 build v10.0.0rc2 (win64)

CPU model: Intel(R) Core(TM) i7-1065G7 CPU @ 1.30GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 16 rows, 30 columns and 90 nonzeros
Model fingerprint: 0x20186c14
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [2e+00, 7e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [6e+01, 2e+02]
Presolve removed 5 rows and 0 columns
Presolve time: 0.02s
Presolved: 11 rows, 35 columns, 65 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    0.0000000e+00   1.610000e+02   0.000000e+00      0s
      15    1.7048900e+03   0.000000e+00   0.000000e+00      0s

Solved in 15 iterations and 0.03 seconds (0.00 work units)
Optimal objective  1.704890000e+03


You are told there is a new policy for transporting **widgets** from production facilities. It is now required that the minimum number of widgets shipped from any production facility to any distribution center needs to be at least 20. 

**Q1.** Write out how the formulation changes using mathematical notation given the new requirement.

\begin{align*} 
x_{p,d} \ge 20, \quad \forall p \in P, d \in D
\end{align*}

**Q2.** Write the changes for **Q1** in gurobipy code in the cell below (no need to run it, unless you want to copy the model here to check).

In [29]:
m.addConstrs(x[p,d] >= 20 for p in production for d in distribution) # think about indices as atrenative way 
m.update()

The initial widget model $m$ represented a single time period (e.g. a week, month, quarter). Suppose we added a time component using a set $T = \{0, 1, 2, 3\}$ representing a quarter of a year. 

**Q3.** Use `addVar()` or `addVars()` to create a decision variable in gurobipy that represents the number of widgets shipped from a production facility to a distribution center for a given time period. 

In [30]:
T=[0,1,2,3]
x = m.addVars(production, distribution, range(4), name = 'prod_ship') # 3d matrix with range(4)

x={}
for p in production:
    for d in ditribution:
        for t in range:
            x[p,d,t]= addVar(name="prod_ship_time")
            
            # reminder use the LP file 
            
        
        

**Q4.** To reference a time period before a given time $t$, we can use $t-1$ as a subscript (since $T$ is a set of integers), but $t-1$ doesn't work when $t = 0$ since $-1$ isn't in $T$. Fill in the ??? in the code below to represent a set of constraints that limits the amount a production facility can produce to `max_prod[p]` over *two consecutive* time periods. 

In [40]:
time_prod_limit = m.addConstrs((gp.quicksum(x[p,d,???] + x[p,d,???] for d in distribution) <= max_prod[p] for p in production for t in ??? if t ???), name = 'time_prod_limit')

SyntaxError: invalid syntax (780851614.py, line 1)

In [ ]:
time_prod_limit = m.addConstrs((gp.quicksum(x[p,d,t] + x[p,d,t-1] for d in distribution) <= max_prod[p] for p in production for t in range(4) if t > 0), name = 'time_prod_limit')